# Python para el análisis de datos
## Introducción a las Ciencias Sociales Computacionales
## Facultad de Ciencias Sociales, Universidad de Buenos Aires, 2023
### Juan Manuel Pérez y Rodolfo Elbert

En esta notebook vamos a analizar datos de las últimas PASO (Agosto 2023), obtenidos del [Observatorio de La Izquierda Diario](https://observatorio.laizquierdadiario.com/consultas). 


Inspirado un poco en este análisis de ["Los Mitos del votante de Milei"](https://medium.com/@VoteBetta/los-mitos-del-votante-de-milei-ae497168535) -- de otro docente de la casa.


## Datos tabulares

En primer lugar, vamos a usar la librería `pandas` para leer los datos de un archivo `csv` y mostrarlos en pantalla.

[pandas](https://pandas.pydata.org/) es una librería de Python para el análisis de datos. En particular, nos permite trabajar con datos tabulares (filas y columnas) de manera muy eficiente y fácil, del estilo de las tablas de Excel. Para quienes tengan experiencia en R, `pandas` es el equivalente a los DataFrame

Un `csv` (comma separated values) es un archivo de texto plano que contiene datos separados por comas. Es un formato muy común para guardar datos tabulares (similar al .xlsx)

In [ ]:
!pip install seaborn pandas statsmodels

In [ ]:
import pandas as pd

df = pd.read_csv('../data/pba_paso_2023.csv')

Vamos a ver algunos elementos de esta tabla.

In [ ]:

df.head(20)

¿Qué columnas tiene? ¿Cuántas filas? ¿Qué tipo de datos tiene cada columna?



In [ ]:
df.info()

¿Cómo vemos qué valores tiene una columna en particular? ¿Y una fila? ¿Qué valores únicos tiene una columna?

Vamos a preguntarle todo esto a ChatGPT.



In [ ]:
# Preguntar a ChatGPT

df["nom_seccion"]

In [ ]:
df["nom_seccion"].value_counts()

Obtener la primer fila

Obtener todas las filas de La Matanza

Obtener todos los votos positivos o blancos

Obtener todos los votos positivos o blancos de la siguiente sección

In [ ]:
seccion = [
    "La Matanza",
    "La Plata",
    "Lomas de Zamora",
    "Merlo",
    "Lanús",
    "Moreno",
    "Morón",
    "Luján",
    "Marcos Paz",
]

df = df[df["nom_seccion"].isin(seccion)]
df = df[(df["nom_tipo_voto"] == "positivo") | (df["nom_tipo_voto"] == "blancos")]


In [ ]:
df

In [ ]:
# Pivot table

df_pivot = df.pivot_table(index="nom_mesa", columns="nom_agrupacion", values="votos", aggfunc="sum")

df_pivot

In [ ]:

# Agrego una columna con el total de votos
df_pivot["total"] = df_pivot.sum(axis=1)


Vamos a usar la librería `matplotlib` para hacer un histograma de la cantidad de votos por mesa.

In [ ]:
from matplotlib import pyplot as plt

plt.hist(df_pivot["total"], bins=40)

Hay algunas mesas con muy pocos votos. Saquémoslas del análisis.

In [ ]:
df_pivot["total"] > 100

In [ ]:
# Saquemos las que tienen menos de 100 votos

df_pivot = df_pivot[df_pivot["total"] > 100]

In [ ]:

renames = {
    "LA LIBERTAD AVANZA": "LLA",
    "FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD": "FIT",
    "UNION POR LA PATRIA": "UxP",
    "JUNTOS POR EL CAMBIO": "JUNTOS",
    "PRINCIPIOS Y VALORES": "PYV",
    "FRENTE LIBER.AR": "LIBER.AR",
    "HACEMOS POR NUESTRO PAIS": "HACEMOS",
    "MOVIMIENTO IZQUIERDA JUVENTUD Y DIGNIDAD": "MIJD",
    "UNION DEL CENTRO DEMOCRATICO": "UCD",
    "FRENTE PATRIOTA FEDERAL": "BIONDINI",
}

# Saco los NaN

df_pivot = df_pivot.dropna()
df_pivot = df_pivot[(df_pivot.T != 0).any()]


# Calculo porcentajes
df_non_normalized = df_pivot.copy()

df_pivot = df_pivot.div(df_pivot.sum(axis=1), axis=0)

# Rename columns

df_pivot = df_pivot.rename(columns=renames)


df_pivot

In [ ]:
# Box plots de los porcentajes

df_pivot.boxplot(figsize=(20, 10))


Hagamos un mapa de calor de las correlaciones entre los votos a las distintas fuerzas.

In [ ]:
corr = df_pivot.corr()

# Plot heatmap

import seaborn as sns

sns.heatmap(
    corr,
    xticklabels=corr.columns.values,
    yticklabels=corr.columns.values,
    cmap="viridis"
)

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np

# Fit linear model with constant for log log model

y = "LLA"
x = "UxP"


df_reg = df_pivot[[x, y]].copy()

df_reg = df_reg[(df_reg[y] > 0.05) & (df_reg[x] > 0.05)]

X = sm.add_constant(df_reg[x])
model = sm.OLS(df_reg[y], X).fit()

print(model.summary())

plt.scatter(df_reg[x], df_reg[y])

plt.plot(X, model.predict(X), color='red', linewidth=3)

plt.xlim(0, df_reg[x].max() + 0.1)
plt.xlabel(x)
plt.ylabel(y)

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np

# Fit linear model with constant for log log model

y = "LLA"
x = "JUNTOS"


df_reg = df_pivot[[x, y]].copy()

df_reg = df_reg[(df_reg[y] > 0.05)]# & (df_reg[x] > 0.05)]

X = sm.add_constant(df_reg[x])
model = sm.OLS(df_reg[y], X).fit()

print(model.summary())

plt.scatter(df_reg[x], df_reg[y])

plt.plot(X, model.predict(X), color='red', linewidth=3)

plt.xlim(0, df_reg[x].max() + 0.1)
plt.xlabel(x)
plt.ylabel(y)

## Resultados 2021

In [ ]:
df_2021 = pd.read_csv("../data/pba.csv")

df_2021.info()

In [ ]:
df_2021_la_matanza = df_2021[df_2021["seccion_nombre"] == "La Matanza"]

df_2021_la_matanza

In [ ]:
df_2021_la_matanza["mesa_id"]

In [ ]:
df[["cod_mesa", "nom_mesa"]]

In [ ]:
df["mesa_id"] = df["cod_mesa"].apply(lambda x: int(x[:-1]))

In [ ]:
df_la_matanza = df[df["nom_seccion"] == "La Matanza"]

In [ ]:
df_2021_la_matanza["mesa_id"].unique()

In [ ]:
df_la_matanza["mesa_id"].unique()

## Ejercicios:

1. Analizar qué pasó con el voto en blanco
2. Buscar qué pasó en mesas donde alguna fuerza sacó 0 votos
3. Intentar reproducir votos 2021 vs votos 2023 (no garantizo que sea posible)
4. Lo que les interese más :-)
